# Samenvoegen en statistieken

In [141]:
base_dir = '/Users/sofiegielis/Documents/BIOMINA/cluster_bookchapter/tutorial'

### Concatenate results

In [144]:
import os
import pandas as pd


def read_results(folder, file):
    
    """
    Read in a TCRex results file as a pandas dataframe.
    
    Args:
    - folder: The folder where the TCRex results file is located
    - file: The name of the TCRex results file
    """
    
    # When reading the data into a dataframe, omit the first 7 lines with meta info
    results = pd.read_csv(os.path.join(folder,file),
                              skiprows=[0, 1, 2, 3, 4, 5, 6], sep='\t')
    return results


def concatenate_data(folder, name):
    
    """
    Concatenate TCRex results from different files into one file.
    
    Args:
    - folder: Path to the directory where the folder with the TCRex results files are located.
    - name: Name of the folder holding all the TCRex results file that need to be concatenated.
    """

    # Get a list of all files in the folder
    files = os.listdir(folder)
    # Remove the hidden DS_store file if present
    if '.DS_Store' in files:
        files.remove('.DS_Store')
        
    # Collect data from all files in a list
    # Emtpy list to collect all data
    results = []
    for fn in files:
        # Read in file fn in dataframe
        data = read_results(folder, fn)
        # Add dataframe to the results list
        results.append(data)
    # Concatenate all dataframes in the results list 
    all_results = pd.concat(results) 

    # Save concatenated dataframe in a new folder
    new_folder = os.path.join(base_dir,'results/parsed_results')
    if not os.path.exists(new_folder):
            os.mkdir(new_folder)
    all_results.to_csv(os.path.join(new_folder,'.'.join([name,'tsv'])),
                       sep='\t', index=False)
    

In [146]:
concatenate_data(os.path.join(base_dir,'results/test'),'filename')

### Get metrics

In [ ]:
def identification_rate(nr_identified, repertoire_size):
    
    """
    Calculate the percentage of epitope-specific TCRs in a repertoire.
    
    Args:
    - nr_identified: The number of identified epitope-specific TCRs 
    - repertoire_size: The number of TCRs in the original repertoire, reported on the TCRex results page
    
    """
    ir = (nr_identified/repertoire_size)*100
    
    return ir


In [ ]:
from scipy import stats as statistics


def enrichment_analysis(nr_identified, repertoire_size, enrichment_threshold):
    
     """
     Calculate the p value of a one sided binomial test.
     
     Args:
     - nr_identified:  The number of identified epitope-specific TCRs 
     - repertoire_size: The number of TCRs in the original repertoire, reported on the TCRex results page
     - enrichment_threshold: Probability of success as defined in a binomial test.
     
     """
    
     p_value = statistics.binom_test(x=nr_identified, n=repertoire_size,
                                     p=enrichment_threshold, alternative='greater')
     
     return p_value
    

In [147]:
# Read the resuls
results = pd.read_csv(os.path.join(base_dir,'results/parsed_results','test.tsv'), sep='\t')

# Calculate the number of identified epitope-specific TCRs 
nr_identified = results.shape[0]  

# Define the repertoire size
repertoire_size = 100000

# Calculate the identification metrics
enrichment_analysis(nr_identified, repertoire_size, 0.001 )
identification_rate(nr_identified, repertoire_size)

0.037

### For multiple epitopes


In [148]:
def calculate_metrics(results, repertoire_size, enrichment_threshold):
    
    """
    Calculate the identification rate and enrichment analysis p value for every epitope in a TCRex results file.
    
    Args:
    - results: Pandas DataFrame containing the TCRex results
    - repertoire_size: The number of TCRs in the original repertoire, reported on the TCRex results page
    - enrichment_threshold: Probability of success as defined in a binomial test.
    
    """
    
    # For every epitope, store the calculated metrics in a dictionary
    metrics = {}
    for epitope in set(results['epitope'].tolist()):
        metrics[epitope] = {}
        # Retrieve all TCRs specific for the epitope
        epitope_data = results[results['epitope'] == epitope]
        # Calculate the number of epitope-specific TCRs
        nr_identified = epitope_data.shape[0]
        # Calculate the required metrics
        metrics[epitope]['identification_rate'] = identification_rate(nr_identified, repertoire_size)
        metrics[epitope]['p_value'] = enrichment_analysis(nr_identified, repertoire_size, enrichment_threshold)
        
    return metrics

    

In [149]:
# Read the resuls
results = read_results(os.path.join(base_dir,'results'),'multiple.tsv')
# Calculate the identification rate and enrichment analysis p value
metrics = calculate_metrics(results,10000,0.0001)
# Organize the metrics in a dataframe
metrics_df = pd.DataFrame.from_dict(metrics)
# Transpose the dataframe
metrics_df = metrics_df.T
metrics_df

,identification_rate,p_value
QYDPVAALF,0.16,1.848163e-14
QIKVRVKMV,0.14,4.484526e-12
YSEHPTFTSQY,0.41,1.041819e-50
NLVPMVATV,27.13,0.000000e+00
IPSINVHHY,0.13,6.317656e-11
VTEHDTLLY,0.96,2.394793e-151
TPRVTGGGAM,0.27,3.391162e-29


#### multiple testing correction

In [150]:
from statsmodels.stats.multitest import multipletests

metrics_df['adjusted_p_value'] = multipletests(metrics_df['p_value'].tolist(), 
              method='fdr_bh', is_sorted=False)[1]

In [151]:
metrics_df

,identification_rate,p_value,adjusted_p_value
QYDPVAALF,0.16,1.848163e-14,2.587428e-14
QIKVRVKMV,0.14,4.484526e-12,5.231947e-12
YSEHPTFTSQY,0.41,1.041819e-50,2.430911e-50
NLVPMVATV,27.13,0.000000e+00,0.000000e+00
IPSINVHHY,0.13,6.317656e-11,6.317656e-11
VTEHDTLLY,0.96,2.394793e-151,8.381774e-151
TPRVTGGGAM,0.27,3.391162e-29,5.934533e-29
